In [21]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
import gc
import numpy as np
from sklearn.datasets import load_files

In [22]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-5Y_bbsUogkAlEz3f-4CflbT40jg-UFJ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-5Y_bbsUogkAlEz3f-4CflbT40jg-UFJ" -O 10_painting.zip && rm -rf /tmp/cookies.txt

--2023-06-27 13:36:41--  https://docs.google.com/uc?export=download&confirm=t&id=1-5Y_bbsUogkAlEz3f-4CflbT40jg-UFJ
Resolving docs.google.com (docs.google.com)... 108.177.112.139, 108.177.112.138, 108.177.112.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.112.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/183kk3ga3m81ditdgpmoba6284mf372p/1687872975000/14533323077134147075/*/1-5Y_bbsUogkAlEz3f-4CflbT40jg-UFJ?e=download&uuid=b01fd61b-e2d3-4d52-9848-3fea677dadca [following]
--2023-06-27 13:36:41--  https://doc-00-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/183kk3ga3m81ditdgpmoba6284mf372p/1687872975000/14533323077134147075/*/1-5Y_bbsUogkAlEz3f-4CflbT40jg-UFJ?e=download&uuid=b01fd61b-e2d3-4d52-9848-3fea677dadca
Resolving doc-00-3c-docs.googleusercontent.com (doc-00-3c-docs.googleusercontent.com)... 142.250.1.13

In [23]:
!unzip -q 10_painting.zip

replace painting/airplane/painting_002_000012.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [24]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ICRO0C8XY9LYR5UPxbHowV1Yxfybkh0D' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ICRO0C8XY9LYR5UPxbHowV1Yxfybkh0D" -O 10_real.zip && rm -rf /tmp/cookies.txt

--2023-06-27 13:44:26--  https://docs.google.com/uc?export=download&confirm=t&id=1ICRO0C8XY9LYR5UPxbHowV1Yxfybkh0D
Resolving docs.google.com (docs.google.com)... 108.177.112.101, 108.177.112.138, 108.177.112.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.112.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ckek6d10n7g5ga16r81a3vjlgojk2aa2/1687873425000/14533323077134147075/*/1ICRO0C8XY9LYR5UPxbHowV1Yxfybkh0D?e=download&uuid=c2272387-8e52-4377-85c3-b33c51decf77 [following]
--2023-06-27 13:44:26--  https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ckek6d10n7g5ga16r81a3vjlgojk2aa2/1687873425000/14533323077134147075/*/1ICRO0C8XY9LYR5UPxbHowV1Yxfybkh0D?e=download&uuid=c2272387-8e52-4377-85c3-b33c51decf77
Resolving doc-0c-3c-docs.googleusercontent.com (doc-0c-3c-docs.googleusercontent.com)... 142.250.1.13

In [25]:
!unzip -q 10_real.zip

replace real/airplane/real_002_000122.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [26]:
real_images = tf.keras.utils.image_dataset_from_directory(
    directory='real/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=((32, 32)),
)

painting_images = tf.keras.utils.image_dataset_from_directory(
    directory='painting/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=((32, 32)),
)

Found 4653 files belonging to 10 classes.
Found 2287 files belonging to 10 classes.


In [ ]:
x_real, y_real = np.concatenate(list(real_images.map(lambda x, y: x))), np.concatenate(list(real_images.map(lambda x, y: y)))
x_paint, y_paint = np.concatenate(list(painting_images.map(lambda x, y: x))), np.concatenate(list(painting_images.map(lambda x, y: y)))

In [28]:
x_real = x_real / 255.0
x_paint = x_paint / 255.0

In [29]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [30]:
x_real_ = x_real.copy()
y_real_ = y_real.copy()

for i in range(17):
    aug_ds = data_augmentation(x_real).numpy()
    x_real_ = np.concatenate([x_real_, aug_ds])
    y_real_ = np.concatenate([y_real_, y_real])

x_paint_ = x_paint.copy()
y_paint_ = y_paint.copy()

for i in range(19):
    aug_ds = data_augmentation(x_paint).numpy()
    x_paint_ = np.concatenate([x_paint_, aug_ds])
    y_paint_ = np.concatenate([y_paint_, y_paint])

In [31]:
input_2 = layers.Input(shape=(32, 32, 3))

y = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_2)
y = layers.MaxPooling2D((2, 2), padding="same")(y)
y = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(y)
y = layers.MaxPooling2D((2, 2), padding="same")(y)
y = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(y)
y = layers.MaxPooling2D((2, 2), padding="same")(y)
y = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(y)
y = layers.MaxPooling2D((2, 2), padding="same")(y)

features_supervised = layers.Flatten()(y)

classification_head = layers.Dense(256, activation='relu')(features_supervised)
classification_head = layers.Dropout(0.4)(classification_head)
classification_head = layers.Dense(128, activation='relu')(classification_head)
classification_head = layers.Dropout(0.2)(classification_head)

classification_head = layers.Dense(10, activation='softmax', name='classification_head')(classification_head)

classifier = Model(input_2, classification_head, name='classifier')
features_model_supervised = Model(input_2, features_supervised, name='features_supervised')

In [32]:
input_1 = layers.Input(shape=(32, 32, 3))

# Encoder
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_1)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

latent = layers.Flatten()(x)

o = GradReverse(0.)(latent)
o = layers.Dropout(0.1)(o)
o = layers.Dense(128, activation='relu')(o)
o = layers.Dense(1, activation='sigmoid', name='discrimination_head')(o)

# Decoder
d = layers.Reshape((4, 4, 64*4))(latent)
d = layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same")(d)
d = layers.BatchNormalization()(d)
d = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(d)
d = layers.BatchNormalization()(d)
d = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(d)

decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same', name='reconstruction_head')(d)

autoencoder = Model(input_1, decoded, name='autoencoder')
encoder = Model(input_1, latent)

In [33]:
classifier.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
)

In [34]:
gc.collect()

1660

In [35]:
classifier.fit(x_real_, y_real_, epochs=30, batch_size=128, shuffle=True, validation_split=0.1)

Epoch 1/30
589/589 [==============================] - 7s 7ms/step - loss: 2.2431 - accuracy: 0.1291 - val_loss: 2.2285 - val_accuracy: 0.1378
Epoch 2/30
589/589 [==============================] - 4s 7ms/step - loss: 2.2312 - accuracy: 0.1360 - val_loss: 2.2247 - val_accuracy: 0.1404
Epoch 3/30
589/589 [==============================] - 4s 6ms/step - loss: 2.2268 - accuracy: 0.1383 - val_loss: 2.2208 - val_accuracy: 0.1486
Epoch 4/30
589/589 [==============================] - 4s 6ms/step - loss: 2.2225 - accuracy: 0.1419 - val_loss: 2.2185 - val_accuracy: 0.1526
Epoch 5/30
589/589 [==============================] - 4s 7ms/step - loss: 2.2186 - accuracy: 0.1474 - val_loss: 2.2126 - val_accuracy: 0.1463
Epoch 6/30
589/589 [==============================] - 4s 6ms/step - loss: 2.2144 - accuracy: 0.1508 - val_loss: 2.2094 - val_accuracy: 0.1535
Epoch 7/30
589/589 [==============================] - 4s 6ms/step - loss: 2.2098 - accuracy: 0.1551 - val_loss: 2.2049 - val_accuracy: 0.1591
Epoch 

In [36]:
autoencoder.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
)

In [37]:
gc.collect()

1534

In [38]:
autoencoder.fit(x_real_, x_real_, epochs=10, batch_size=128, shuffle=True, validation_split=0.1)

Epoch 1/10
589/589 [==============================] - 11s 14ms/step - loss: 0.4549 - val_loss: 0.4387
Epoch 2/10
589/589 [==============================] - 7s 12ms/step - loss: 0.4296 - val_loss: 0.4294
Epoch 3/10
589/589 [==============================] - 8s 13ms/step - loss: 0.4248 - val_loss: 0.4251
Epoch 4/10
589/589 [==============================] - 8s 13ms/step - loss: 0.4220 - val_loss: 0.4293
Epoch 5/10
589/589 [==============================] - 7s 13ms/step - loss: 0.4200 - val_loss: 0.4203
Epoch 6/10
589/589 [==============================] - 8s 13ms/step - loss: 0.4183 - val_loss: 0.4198
Epoch 7/10
589/589 [==============================] - 8s 13ms/step - loss: 0.4172 - val_loss: 0.4194
Epoch 8/10
589/589 [==============================] - 7s 12ms/step - loss: 0.4162 - val_loss: 0.4229
Epoch 9/10
589/589 [==============================] - 8s 13ms/step - loss: 0.4153 - val_loss: 0.4190
Epoch 10/10
589/589 [==============================] - 7s 12ms/step - loss: 0.4149 - val_l

In [39]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers

@tf.custom_gradient
def grad_reverse(x,l):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy * l, None
    return y, custom_grad

class GradReverse(Layer):
    #def __init__(self, l, **kwargs):
    def __init__(self, l):
        super().__init__()
        self.l = tf.Variable(l, trainable=False)

    def call(self, x):
        return grad_reverse(x,self.l)

    def get_config(self):
        return {'l': self.l.numpy()}

In [67]:
input_3 = layers.Input(shape=(32, 32, 3))

# features = tf.keras.layers.Concatenate(axis=1)([encoder(input_3), features_model_supervised(input_3)])
features = features_model_supervised(input_3)
o = GradReverse(0.)(features)
o = layers.Dropout(0.3)(o)
o = layers.Dense(128, activation='relu')(o)
o = layers.Dense(1, activation='sigmoid', name='discrimination_head')(o)

model = Model(input_3, o, name='ahh')
features = Model(input_3, features)

In [68]:
model.summary()

Model: "ahh"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 features_supervised (Functi  (None, 256)              65568     
 onal)                                                           
                                                                 
 grad_reverse_7 (GradReverse  (None, 256)              1         
 )                                                               
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 discrimination_head (Dense)  (None, 1)                129     

In [69]:
combined_images = tf.concat([x_real_[:40000], x_paint_[:40000]], axis=0)

labels = tf.concat(
    [tf.ones((40000, 1)), tf.zeros((40000, 1))], axis=0
)

In [70]:
model.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [71]:
h = model.fit(
    x=combined_images,
    y=labels,
    epochs=25,
    batch_size=128,
    shuffle=True,
    validation_split=0.1
)

Epoch 1/25
563/563 [==============================] - 7s 7ms/step - loss: 0.6144 - accuracy: 0.6465 - val_loss: 0.6411 - val_accuracy: 0.6628
Epoch 2/25
563/563 [==============================] - 4s 7ms/step - loss: 0.5920 - accuracy: 0.6713 - val_loss: 0.6610 - val_accuracy: 0.6409
Epoch 3/25
563/563 [==============================] - 4s 6ms/step - loss: 0.5813 - accuracy: 0.6802 - val_loss: 0.6740 - val_accuracy: 0.6290
Epoch 4/25
563/563 [==============================] - 3s 6ms/step - loss: 0.5715 - accuracy: 0.6916 - val_loss: 0.5554 - val_accuracy: 0.7464
Epoch 5/25
563/563 [==============================] - 4s 7ms/step - loss: 0.5667 - accuracy: 0.6942 - val_loss: 0.6335 - val_accuracy: 0.6695
Epoch 6/25
563/563 [==============================] - 4s 7ms/step - loss: 0.5600 - accuracy: 0.7007 - val_loss: 0.6109 - val_accuracy: 0.6975
Epoch 7/25
563/563 [==============================] - 3s 6ms/step - loss: 0.5562 - accuracy: 0.7051 - val_loss: 0.5430 - val_accuracy: 0.7586
Epoch 

In [72]:
gc.collect()

3191

In [73]:
features_train = features.predict(combined_images)

2500/2500 [==============================] - 4s 2ms/step


In [74]:
def onehot_to_label(x):
    labels = []
    for i in range(x.shape[0]):
        labels.append(np.argmax(x[i]))
    return labels

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(features_train, onehot_to_label(labels))

RandomForestClassifier(max_depth=10, random_state=0)

In [76]:
features_test = features(x_paint)

In [77]:
y_pred = clf.predict(features_test)

In [78]:
accuracy_score(onehot_to_label(y_paint), y_pred)

0.09269785745518146